In [167]:
from IPython.display import Image
from IPython.core.display import HTML 
#from __future__ import print_func  tion, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import json
from sklearn.model_selection import train_test_split
Image(url= "https://cdn-images-1.medium.com/max/1600/1*UkI9za9zTR-HL8uM15Wmzw.png")

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 500)


In [168]:
# the columns use for predictions
# NOTE:
# When adding more features
# make sure make the model in a differnt path because it screws with the checkpoints
saved_model = "./6_col_checkpoints/"
features = ['posX', 'posY', 'posZ', 'velX', 'velY', 'velZ']
col_numbers = len(features)


In [169]:
# copying the dataframe so I have a before and after
df_cir_hor_cclk = pd.read_csv('c:/Users/Franz Lom/Documents/TensorFlow/gesture_recognition/Data/extracted_data/cir_hor_cclk.csv') # saving circle horizontal counter clockwise csv as df
df_v_shape = pd.read_csv('c:/Users/Franz Lom/Documents/TensorFlow/gesture_recognition/Data/extracted_data/v_shape.csv')
#df_cir_hor_cclk['shape'] = 1
#df_v_shape['shape'] = 2

df_cir = df_cir_hor_cclk.copy()
df_v = df_v_shape.copy()

df_cir['shape'] = 1
df_v['shape'] = 2
list(df_cir_hor_cclk)


['tester',
 'trial',
 'rightHanded',
 'timestamp',
 'posX',
 'posY',
 'posZ',
 'posRelX',
 'posRelY',
 'posRelZ',
 'velX',
 'velY',
 'velZ',
 'quatW',
 'quatX',
 'quatY',
 'quatZ',
 'accRelX',
 'accRelY',
 'accRelZ',
 'accAbsX',
 'accAbsY',
 'accAbsZ']

In [170]:
def pad_features(not_pad_features):
    """
    A function that returns a padded array with a size 300
    :param not_pad_features: 
    :return: an np.array of padded vector with the length of  300 
    """
    
    list_pad_vec = list()
    feature_len = [len(s) for s in not_pad_features]
    if max(feature_len) > 300:
        raise Exception('the longest stroke is greater than 300. len: [}'
                        .format(max(feature_len)))
    padding = np.zeros((300, col_numbers))
    
    for x in not_pad_features:
        padding[:x.shape[0], :x.shape[1]] = x 
        list_pad_vec.append(padding)
        padding = np.zeros((300, col_numbers))
        
    return(np.array(list_pad_vec))

In [171]:
def group_drawn_shape(grouped):
    """
    
    :param grouped: grouped value from the dataframe, ['trial'. 'tester']
    :return: np.array of grouped vectors
    """
    source = [x for x in grouped]
    list_vec = list()
    for v in source:
        df = v[1]
        df = df[features]
        x = df.values
        list_vec.append(x)
    return np.array(list_vec)

In [173]:
cir_features = group_drawn_shape(df_cir.groupby(['trial', 'tester']))
v_features = group_drawn_shape(df_v.groupby(['trial', 'tester']))

cir_stroke_length = [len(stroke) for stroke in cir_features]
cir_stroke = [stroke for stroke in cir_features]
print(cir_features.shape)
pad_cir_features = pad_features(cir_features)
pad_v_features = pad_features(v_features)

(280,)


In [174]:
print(pad_cir_features.shape)
print(pad_v_features.shape)

#combine the padded data so we have one data to work with
data = np.vstack((pad_cir_features, pad_v_features))
data.shape


(280, 300, 6)
(280, 300, 6)


(560, 300, 6)

In [175]:
# labels are one hot encoded
horizontal_circle_label = np.zeros((280,2))
v_shape_label = np.zeros((280,2))

horizontal_circle_label[:, 0] = 1
v_shape_label[:, 1] = 1

labels = np.vstack((horizontal_circle_label, v_shape_label))
labels[275:285]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [176]:
horizontal_circle_label = np.zeros(280)
v_shape_label = np.zeros(280)
v_shape_label[:] = 1
labels = np.concatenate([horizontal_circle_label, v_shape_label])
labels[275:285]

array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

In [177]:
# split the data to training training and testing data.
# x = features
# y = labels
x_train, x_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.20, random_state=42)

print(x_train.shape)
print(y_train.shape)

(448, 300, 6)
(448,)


In [194]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},          # this are the features
    y=np.int32(y_train),       # this are the classification (gesture)
    num_epochs=1,
    shuffle=False
)
print(type(y_train))
y_train[0:10]

<class 'numpy.ndarray'>


array([1., 1., 0., 0., 1., 0., 1., 1., 1., 1.])

In [207]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=np.int32(y_test),
    num_epochs=1,
    shuffle=False
)

In [179]:
some_gesture =x_test[0:9]
some_gesture_labels = y_test[0:9]


In [180]:
features_x = tf.feature_column.numeric_column(key="x", shape=(300, col_numbers))
feature_columns = [features_x]

In [213]:
def model_fn(features, labels, mode, params):
    """
    The neural network
    fully connected dense 1 layer convolutional neural network
    :param features: (x) argument from the input_fn
    :param labels:   (y) argument from the input_fn
    :param mode:      either TRAIN, EVAL or PREDICT
    :param params:    learning rate
    :return: 
    """
    print(labels[0:5])
    print('\n', labels.shape)
    x = features["x"]
    
    net = tf.reshape(x, [-1, col_numbers, 300, 2])
    
    # first convolutional layer
    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=16, kernel_size=5,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)
    
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

    #net = tf.contrib.layers.flatten(net)
    net = tf.layers.flatten(net)
    
    net = tf.layers.dense(inputs=net, name='layer_fc1',
                          units=128, activation=tf.nn.relu)
    
    net = tf.layers.dense(inputs=net, name='layer_fc2',
                          units=10)
    
    logits = net
    
    y_pred = tf.nn.softmax(logits=logits)
    y_pred_cls = tf.argmax(y_pred, axis=1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)
        loss = tf.reduce_mean(cross_entropy)
        
        optimiser = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])
        
        train_op = optimiser.minimize(
            loss=loss, global_step=tf.train.get_global_step()
        )
        
        metrics = \
            {
                "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
            }
        
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics
        )
    return spec


In [210]:
params = {"learning_rate": 1e-4}

In [211]:
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir="./custom_checkpoints_3/")

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_train_distribute': None, '_evaluation_master': '', '_task_type': 'worker', '_global_id_in_cluster': 0, '_service': None, '_save_checkpoints_secs': 600, '_session_config': None, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000011476BEB240>, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_task_id': 0, '_device_fn': None, '_model_dir': './custom_checkpoints_3/', '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_master': '', '_tf_random_seed': None}


In [212]:
model.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.


Tensor("strided_slice:0", shape=(?,), dtype=int32)

 (?,)


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./custom_checkpoints_3/model.ckpt-0


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into ./custom_checkpoints_3/model.ckpt.


InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [64,10] and labels shape [128]
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_DOUBLE, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](layer_fc2/BiasAdd, fifo_queue_DequeueUpTo:2)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "C:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  Fil

In [208]:
result = model.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.


Tensor("strided_slice:0", shape=(?,), dtype=int32)

 (?,)


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2018-11-12-22:12:01


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./custom_checkpoints_3/model.ckpt-0


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [56,10] and labels shape [112]
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_DOUBLE, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](layer_fc2/BiasAdd, fifo_queue_DequeueUpTo:2)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "C:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  Fil